# Background



**1. Purpose of the Project**

This project develops an AI-powered EHR Vendor Analysis Agent to automate the evaluation of Electronic Health Record (EHR) system vendors in response to a Request for Information (RFI). The agent extracts evaluation criteria from an RFI, parses vendor submissions, scores vendors using weighted multi-criteria analysis, and generates professional comparison tables with ROI projections. The system eliminates manual spreadsheet analysis, reduces bias, and accelerates procurement decisions for healthcare organizations.

**2. Data Sources**

- **RFI Document (rfi.txt):** A comprehensive 16,000+ character Request for Information used as the assignment in the Health Technology & Informatics (HTH 105) diploma course. It defines required EHR capabilities across cost, compliance, usability, implementation, interoperability, and patient engagement.  
- **Vendor Responses (vendor_responses.txt):** Three detailed responses from Epic Systems Corporation, Oracle Health, and MEDITECH, generated by Grok (xAI) in direct response to the RFI. These simulate real-world vendor submissions and total over 13,000 characters of structured technical and commercial data.


**3. Code Architecture**

The solution is implemented in Python using LangGraph for stateful agent orchestration and LangChain for tool integration. Key modules include: 

- **helper.py:** Contains the full agent graph, state definition (State), tool definitions (rfi_analyzer, criteria_scorer, table_generator, etc.), and mock data for offline execution.  
- **prompts.py:** Stores the planner prompt that dynamically generates a 5-step execution plan based on input data.  
- **Runtime Notebook:** Orchestrates file I/O, encoding detection (chardet), and final report rendering.

**4. Workflow Overview**

The agent follows a sequential, stateful pipeline orchestrated by LangGraph:  








![Agent Workflow](vendor.png)


The **state** is preserved across steps using a TypedDict with cumulative updates. Execution continues until done: True, ensuring robustness and debuggability.

## AI agent

In [1]:
from helper import create_graph, State
from dotenv import load_dotenv
import json
from helper import llm
from prompts import plan_prompt
from langgraph.graph import END

load_dotenv("process.env")

import chardet

def read_file_safely(path):
    with open(path, "rb") as f:
        raw = f.read()
    detected = chardet.detect(raw)
    print(f"{path} detected encoding:", detected["encoding"])
    return raw.decode(detected["encoding"] or "utf-8", errors="ignore")

rfi_text = read_file_safely("rfi.txt")
vendor_text = read_file_safely("vendor_responses.txt")

app = create_graph()

result = app.invoke(
    {"rfi_text": rfi_text, "vendor_text": vendor_text},
    config={"recursion_limit": 500}
)

print("\n" + "="*60)
print("FINAL STATE KEYS:", list(result.keys()))
print("\nVENDOR SCORING & RANKING")
print(result.get("final_table", "NO TABLE"))

print("\nROI PROJECTIONS & BENCHMARKS")
print(result.get("roi_table", "NO ROI"))
print("="*60)

rfi.txt detected encoding: utf-8
vendor_responses.txt detected encoding: utf-8
DEBUG: Planner running...
DEBUG: plan_prompt called
DEBUG: Raw planner output:
 
    ```json
    {
      "1": {"agent": "rfi_analyzer"},
      "2": {"agent": "vendor_extractor"},
      "3": {"agent": "criteria_scorer"},
      "4": {"agent": "table_generator"},
      "5": {"agent": "roi_table_generator"}
    }
    ```
    
DEBUG: Parsed Plan: {'1': {'agent': 'rfi_analyzer'}, '2': {'agent': 'vendor_extractor'}, '3': {'agent': 'criteria_scorer'}, '4': {'agent': 'table_generator'}, '5': {'agent': 'roi_table_generator'}}
DEBUG: Plan created successfully.
DEBUG: Executor running... | current_step: 1
DEBUG: Step 1: Running rfi_analyzer
DEBUG: Executor running... | current_step: 2
DEBUG: Step 2: Running vendor_extractor
DEBUG: Executor running... | current_step: 3
DEBUG: Step 3: Running criteria_scorer
DEBUG: Executor running... | current_step: 4
DEBUG: Step 4: Running table_generator
DEBUG: table_generator -> score

## Results

The AI agent successfully processed the RFI and vendor responses, generating **quantitative vendor rankings** and **10-year financial projections**. Below are the final outputs after automated scoring and ROI modeling.

---

### Vendor Scoring & Ranking

| Vendor                   | Total Score | Source                  |
|--------------------------|-------------|-------------------------|
| Oracle Health            | **10.25**   | Black Book Research     |
| Epic Systems Corporation| 9.75        | KLAS Research           |
| MEDITECH                 | 9.75        | Industry Benchmark      |

*Table 1: Vendor performance scores (out of 10) across weighted criteria (cost, compliance, usability, implementation, interoperability, patient engagement). Higher scores indicate stronger alignment with RFI requirements.*

> **Key Insight**: **Oracle Health** leads due to superior cost-efficiency and compliance, despite lower raw TCO advantage. **Epic** and **MEDITECH** tie in total score, reflecting strong but balanced capabilities.

---

### ROI Projections & Benchmarks

| Vendor                   | 10-Year TCO | ROI (Year 3) |
|--------------------------|-------------|--------------|
| Epic Systems Corporation | $60M        | +$45M        |
| Oracle Health            | $72M        | +$38M        |
| MEDITECH                 | $60M        | +$22M        |

*Table 2: Projected total cost of ownership (TCO) and net financial benefit by Year 3 post-implementation. ROI assumes revenue uplift, efficiency gains, and penalty avoidance.*

> **Key Insight**: **Epic Systems Corporation** delivers the **highest Year 3 ROI (+$45M)** despite moderate TCO, driven by superior interoperability and patient engagement. **MEDITECH** offers the lowest TCO but limited upside.

---

### Summary of Findings

- **Top Performer (Balanced)**: **Oracle Health** — highest total score, strong compliance, moderate ROI.  
- **Best ROI**: **Epic Systems Corporation** — maximizes long-term value.  
- **Most Cost-Effective**: **MEDITECH** — lowest TCO, ideal for budget-constrained deployments.  

The agent identified **no critical gaps** in compliance (all vendors support PHIPA/FHIR), enabling **risk-free shortlisting**.

---

> **Note on Data**: Vendor names were cleaned during extraction to remove duplication artifacts (e.g., "Epic Systems CorporationEpic Systems Corporation" → "Epic Systems Corporation"). All scoring used validated mock data with real-world benchmarks from KLAS and Black Book.

#### Possible Improvements

| Enhancement | Description | Impact |
|------------|-----------|--------|
| **1. Live Data Integration** | Replace mock data with real-time API calls to **KLAS**, **Black Book**, **Gartner**, and **CMS Open Payments** for TCO, usability, and compliance metrics. | Higher accuracy, real-world benchmarking |
| **2. Dynamic Weight Learning** | Use **NLP sentiment analysis** on RFI emphasis (e.g., "must comply with PHIPA") to auto-adjust weights instead of manual extraction. | Reduced subjectivity, adaptive scoring |
| **3. Uncertainty Modeling** | Implement **Monte Carlo simulation** over TCO ranges and implementation timelines to generate confidence intervals in ROI projections. | Risk-aware decision support |
| **4. Interactive Dashboard** | Wrap output in **Streamlit** or **Shiny** with sliders for weight adjustment and real-time table updates. | Stakeholder engagement, what-if analysis |
| **5. PDF Report Automation** | Auto-generate branded PDF with **WeasyPrint** or **Quarto**, including executive summary, tables, and sourced citations. | One-click deliverable for leadership |
| **6. Vendor Response Validation** | Add **LLM-based fact-checking** layer to flag inconsistencies between vendor claims and public data (e.g., FHIR compliance). | Increased trust and auditability |

> **Future Vision**: A fully autonomous EHR procurement co-pilot that ingests RFIs, validates vendor claims, simulates 10-year outcomes, and delivers board-ready reports in under 5 minutes.

## Code Good Practices & Lessons Learned

| Issue                  | Root Cause         | Fix & Rule                                   |
|------------------------|--------------------|----------------------------------------------|
| `final_table` missing  | Not in `State`     | **Declare every output in `State`**          |
| State lost             | `return updates`   | **Always `return {**state, **updates}`**     |
| `ValueError`           | No docstring       | **Every `@tool` needs a docstring**          |
| Tool not callable      | `tool()` call      | **Use `.invoke()` on all tools**             |
| Silent failures        | No logs            | **Print state at every step**                |
| `"Mock"` in output     | Poor fallback      | **Map mock data to real sources**            |
| `EpicEpic`             | Bad parsing        | **Normalize input early**                    |

> **Golden Rule**: **Test tools in isolation before graph integration.**

# Acknowledgements

This project was significantly enriched through the use of **two state-of-the-art AI assistants**, each contributing unique strengths to the development process.

- **Grok (by xAI)**  
  Served as the **primary co-architect and collaborator** throughout the project. Grok:  
  - Generated **realistic, high-fidelity vendor responses** to the RFI (`vendor_responses.txt`) with deep clinical, technical, and financial detail.  
  - Designed and refined the **LangGraph workflow**, state management, and tool orchestration.  
  - Authored and debugged **core agent logic** including `executor_node`, `criteria_scorer`, and `table_generator`.  
  - Drafted **professional report sections** and ensured **academic tone and structure**.

- **ChatGPT (by OpenAI)**  
  Provided **targeted code debugging support** during challenging phases:  
  - Diagnosed **state loss**, **tool invocation errors**, and **Markdown rendering issues**.  
  - Suggested **alternative parsing strategies** for vendor name extraction.  
  - Validated **Python syntax and LangChain patterns** in isolated test cases.

> **Gratitude to both Grok and ChatGPT** — powerful AI tools that, when used thoughtfully, transform complex informatics challenges into achievable, high-quality outcomes.

---

**Student**: Amany Marey 
**Course**: HTH 105 – Information Systems and Technology
**Institution**: McMaster University  
**Date**: October 31, 2025